# Event-Based Anomaly Detection: CPI → HY OAS & Unemployment Events

> **📚 Detailed Methodology**: See [`docs/cpi_hy_unemployment_methodology.md`](../docs/cpi_hy_unemployment_methodology.md)

## Overview

Detect large moves in target variables around macro announcements:

| Model | Trigger | Target | 
|-------|---------|--------|
| CPI → HY OAS | CPI release | High-Yield spread change |
| Unemp → 10Y | Jobs report | 10Y yield change |
| Unemp → HY | Jobs report | HY spread change |
| Unemp → VIX | Jobs report | VIX change (note: absolute points, not relative) |

## Quick Reference

| Step | Method | Details in Doc |
|------|--------|----------------|
| Dataset | Event-level, t-1 features only | Section 2-3 |
| Split | Chronological 70/30 | Section 4 |
| "Large" threshold | 85th pctl of train |Δ| | Section 5 |
| Model | RF, class_weight={F:1, T:50} | Section 6 |
| Deployment threshold | TimeSeriesSplit CV, median | Section 7 |
| Ex-post diagnostics | FN=0, FN≤1, FN<1% | Section 8 |

## Key Design Principles

- **Cost-sensitive**: Prioritize catching all large moves (FN=0)
- **Time-based splits**: No future data leakage
- **Training-only thresholds**: Large move threshold AND deployment threshold from train only
- **Two types of threshold analysis**:
  - **CV threshold** (Section 7): What we'd deploy live
  - **Ex-post scenarios** (Section 8): Diagnostics on test set tradeoffs


In [ ]:
# Methodology documentation: docs/cpi_hy_unemployment_methodology.md

import sys
from pathlib import Path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix, classification_report

from src.data.merge_panel import build_fed_panel
from src.data.inflation_announcements_loader import load_inflation_announcements
from src.data.fred_loader import load_series
from src.utils.logging_utils import get_logger, setup_logging

setup_logging()
logger = get_logger(__name__)

import warnings
warnings.filterwarnings('ignore')

print('Imports complete. See docs/cpi_hy_unemployment_methodology.md for full methodology.')


---

# Part 1: CPI → HY OAS Edge

> See [methodology doc](../docs/cpi_hy_unemployment_methodology.md) sections 2-3 for feature engineering details.

Detect large HY spread moves around CPI releases.


In [ ]:
def _get_prev_month(period: str) -> str:
    """Get previous month's period string (YYYY-MM format)."""
    year, month = map(int, period.split('-'))
    if month == 1:
        return f"{year - 1}-12"
    else:
        return f"{year}-{month - 1:02d}"


def build_cpi_hy_dataset(
    panel: pd.DataFrame,
    cpi_announcements: pd.DataFrame,
    window_days: int = 0,
    min_history_days: int = 30,
    yield_vol_window: int = 20
) -> pd.DataFrame:
    """
    Build event-level dataset for CPI → HY OAS large move detection.
    
    Args:
        panel: Daily FRED panel with columns including hy_oas, y_2y, y_10y, etc.
        cpi_announcements: CPI release calendar with release_date, data_period
        window_days: Days after announcement to measure change (0 = same day)
        min_history_days: Minimum days of history required
        yield_vol_window: Rolling window for yield volatility calculation
    
    Returns:
        DataFrame with one row per CPI event, containing features and HY OAS change
    """
    # Check required columns in panel
    required_cols = ['hy_oas', 'y_2y', 'y_10y', 'slope_10y_2y', 'fed_funds', 'unemployment', 'cpi']
    missing_cols = [c for c in required_cols if c not in panel.columns]
    if missing_cols:
        raise ValueError(f"Need {missing_cols} in the panel for CPI→HY model.")
    
    # Ensure date columns are datetime
    panel = panel.copy()
    panel['date'] = pd.to_datetime(panel['date'])
    panel = panel.sort_values('date').reset_index(drop=True)
    
    cpi_announcements = cpi_announcements.copy()
    cpi_announcements['release_date'] = pd.to_datetime(cpi_announcements['release_date'])
    
    # Load raw CPI for MoM calculation
    try:
        cpi_raw = load_series("CPIAUCSL")
        cpi_raw = cpi_raw.reset_index()
        cpi_raw.columns = ['date', 'cpi_value']
        cpi_raw['date'] = pd.to_datetime(cpi_raw['date'])
        cpi_raw['year_month'] = cpi_raw['date'].dt.to_period('M')
        cpi_monthly = cpi_raw.groupby('year_month')['cpi_value'].last().reset_index()
        cpi_monthly['data_period'] = cpi_monthly['year_month'].astype(str)
        cpi_dict = dict(zip(cpi_monthly['data_period'], cpi_monthly['cpi_value']))
    except Exception as e:
        logger.warning(f"Could not load raw CPI: {e}. Using panel values.")
        cpi_dict = {}
    
    events = []
    
    for idx, ann in cpi_announcements.iterrows():
        ann_date = ann['release_date']
        data_period = ann['data_period']
        
        # Find panel row for announcement date
        ann_day = panel[panel['date'] == ann_date]
        if ann_day.empty:
            # Try nearby dates (weekend/holiday adjustment)
            for offset in [1, -1, 2, -2, 3, -3]:
                candidate = ann_date + pd.Timedelta(days=offset)
                ann_day = panel[panel['date'] == candidate]
                if not ann_day.empty:
                    ann_date = candidate
                    break
        
        if ann_day.empty:
            continue
        
        ann_row = ann_day.iloc[0]
        
        # Get previous trading day
        prev_days = panel[panel['date'] < ann_date]
        if len(prev_days) < min_history_days:
            continue
        prev_row = prev_days.iloc[-1]
        
        # Get HY OAS before and after
        hy_before = prev_row.get('hy_oas')
        
        if window_days == 0:
            hy_after = ann_row.get('hy_oas')
        else:
            future_date = ann_date + pd.Timedelta(days=window_days)
            future_rows = panel[panel['date'] >= future_date]
            if future_rows.empty:
                continue
            hy_after = future_rows.iloc[0].get('hy_oas')
        
        if pd.isna(hy_before) or pd.isna(hy_after):
            continue
        
        hy_change = hy_after - hy_before
        
        # Compute CPI surprise (MoM)
        current_cpi = cpi_dict.get(data_period, ann_row.get('cpi'))
        prev_month = _get_prev_month(data_period)
        prev_cpi = cpi_dict.get(prev_month)
        
        if prev_cpi and current_cpi and prev_cpi > 0:
            cpi_shock_mom = ((current_cpi - prev_cpi) / prev_cpi) * 100
        else:
            cpi_shock_mom = np.nan
        
        # Calculate yield volatility (trailing window)
        recent = prev_days.tail(yield_vol_window)
        if len(recent) > 5:
            y10_vals = recent['y_10y'].dropna().values
            if len(y10_vals) > 5:
                yield_volatility = np.std(np.diff(y10_vals))
            else:
                yield_volatility = np.nan
        else:
            yield_volatility = np.nan
        
        event = {
            'date': ann_date,
            'data_period': data_period,
            # CPI features
            'cpi_shock_mom': cpi_shock_mom,
            'cpi_shock_abs': abs(cpi_shock_mom) if pd.notna(cpi_shock_mom) else np.nan,
            # Background features (t-1)
            'yield_volatility': yield_volatility,
            'slope_10y_2y': prev_row.get('slope_10y_2y'),
            'fed_funds': prev_row.get('fed_funds'),
            'unemployment': prev_row.get('unemployment'),
            'hy_oas_before': hy_before,
            'y_10y_before': prev_row.get('y_10y'),
            # Stress indicators
            'stlfsi': prev_row.get('stlfsi', np.nan),
            'vix': prev_row.get('vix', np.nan),
            # Target
            'hy_oas_change': hy_change,
            'hy_oas_change_abs': abs(hy_change),
        }
        events.append(event)
    
    df = pd.DataFrame(events)
    if df.empty:
        logger.warning("No events created. Check data availability.")
        return df
    
    df = df.dropna(subset=['hy_oas_change'])
    df = df.sort_values('date').reset_index(drop=True)
    
    logger.info(f"Built CPI→HY dataset: {len(df)} events from {df['date'].min().date()} to {df['date'].max().date()}")
    return df


In [ ]:
# Load data
print("Loading panel data...")
panel = build_fed_panel()
print(f"Panel shape: {panel.shape}")
print(f"Panel columns: {list(panel.columns)}")

print("\nLoading CPI announcements...")
cpi_announcements = load_inflation_announcements()
print(f"CPI announcements: {len(cpi_announcements)}")


In [ ]:
# Build CPI → HY dataset
cpi_hy_df = build_cpi_hy_dataset(panel, cpi_announcements, window_days=0)
print(f"\nCPI→HY Events: {len(cpi_hy_df)}")
print(f"Date range: {cpi_hy_df['date'].min().date()} to {cpi_hy_df['date'].max().date()}")
cpi_hy_df.head(10)


In [ ]:
# Explore HY OAS changes distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

ax1 = axes[0]
ax1.hist(cpi_hy_df['hy_oas_change'].dropna(), bins=50, alpha=0.7, edgecolor='black')
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax1.set_xlabel('HY OAS Change (pp)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of HY OAS Changes Around CPI Releases')

ax2 = axes[1]
ax2.scatter(cpi_hy_df['cpi_shock_mom'], cpi_hy_df['hy_oas_change'], alpha=0.5)
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax2.axvline(x=0, color='red', linestyle='--', alpha=0.5)
ax2.set_xlabel('CPI Shock MoM (%)')
ax2.set_ylabel('HY OAS Change (pp)')
ax2.set_title('CPI Shock vs HY OAS Change')

ax3 = axes[2]
ax3.scatter(cpi_hy_df['hy_oas_before'], cpi_hy_df['hy_oas_change'], alpha=0.5)
ax3.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax3.set_xlabel('HY OAS Before (pp)')
ax3.set_ylabel('HY OAS Change (pp)')
ax3.set_title('HY OAS Level vs Change')

plt.tight_layout()
plt.show()

print("\nHY OAS Change Statistics:")
print(cpi_hy_df['hy_oas_change'].describe())


## Train/Test Split and Threshold
> See [methodology doc](../docs/cpi_hy_unemployment_methodology.md) sections 4-5.


In [ ]:
def compute_large_move_threshold_from_train(
    df: pd.DataFrame,
    target_col: str,
    percentile: float = 90.0
) -> float:
    """
    Compute 'large move' threshold from training data only.
    
    Args:
        df: Training DataFrame
        target_col: Column name for the target change (e.g., 'hy_oas_change')
        percentile: Percentile of |change| to use as threshold (e.g., 90 = top 10%)
    
    Returns:
        Threshold value in same units as target
    """
    abs_changes = df[target_col].abs().dropna()
    threshold = np.percentile(abs_changes, percentile)
    n_large = (abs_changes >= threshold).sum()
    
    logger.info(f"Large move threshold ({percentile}th percentile): {threshold:.4f}")
    logger.info(f"Large moves in training: {n_large} ({n_large/len(abs_changes)*100:.1f}%)")
    
    return threshold


In [ ]:
# Chronological train/test split (70/30)
n_total = len(cpi_hy_df)
n_test = int(n_total * 0.30)
n_train = n_total - n_test

train_df = cpi_hy_df.iloc[:n_train].copy()
test_df = cpi_hy_df.iloc[n_train:].copy()

print("DATA SPLITS (chronological)")
print("="*60)
print(f"Train: {len(train_df)} events ({train_df['date'].min().date()} to {train_df['date'].max().date()})")
print(f"Test:  {len(test_df)} events ({test_df['date'].min().date()} to {test_df['date'].max().date()})")


In [ ]:
# Compute large move threshold from TRAINING DATA ONLY
# Using 85th percentile of |HY OAS change| as "large" (more events for better CV)
LARGE_HY_THRESHOLD = compute_large_move_threshold_from_train(
    train_df, 'hy_oas_change', percentile=85.0
)

print(f"\nLARGE HY OAS THRESHOLD: {LARGE_HY_THRESHOLD:.4f} percentage points")
print(f"(Movements above {LARGE_HY_THRESHOLD*100:.1f} bps will be labeled as 'large')")


In [ ]:
# Create binary labels
train_df['is_large_hy_move'] = train_df['hy_oas_change'].abs() >= LARGE_HY_THRESHOLD
test_df['is_large_hy_move'] = test_df['hy_oas_change'].abs() >= LARGE_HY_THRESHOLD

print("LABEL DISTRIBUTION")
print("-"*40)
print(f"Train large moves: {train_df['is_large_hy_move'].sum()} ({train_df['is_large_hy_move'].mean()*100:.1f}%)")
print(f"Test large moves:  {test_df['is_large_hy_move'].sum()} ({test_df['is_large_hy_move'].mean()*100:.1f}%)")


## Model Training
> See [methodology doc](../docs/cpi_hy_unemployment_methodology.md) section 6 for cost-sensitive RF details.


In [ ]:
def train_cost_sensitive_model(
    train_df: pd.DataFrame,
    feature_cols: list,
    target_col: str,
    class_weight_positive: int = 50,
    n_cv_splits: int = 5,
    model_type: str = 'random_forest'
) -> dict:
    """
    Train a cost-sensitive classifier using time-series cross-validation.
    
    Prioritizes FN=0 (never miss a large move) over minimizing FP.
    
    Args:
        train_df: Training DataFrame
        feature_cols: List of feature column names
        target_col: Binary target column name
        class_weight_positive: Weight for positive class (large moves)
        n_cv_splits: Number of time series CV splits
        model_type: 'random_forest' or 'gradient_boosting'
    
    Returns:
        Dictionary with model, scaler, medians, threshold, and CV metrics
    """
    # Prepare features
    X = train_df[feature_cols].copy()
    y = train_df[target_col].values
    
    # Compute medians for imputation (from training data only)
    medians = {col: X[col].median() for col in feature_cols}
    X = X.fillna(medians)
    
    # Fit scaler on training data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Time series CV for threshold selection
    tscv = TimeSeriesSplit(n_splits=n_cv_splits)
    cv_thresholds = []
    cv_fp_rates = []
    
    print(f"TIME SERIES CROSS-VALIDATION ({n_cv_splits} folds)")
    print("="*60)
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_scaled)):
        X_tr = X_scaled[train_idx]
        X_val = X_scaled[val_idx]
        y_tr = y[train_idx]
        y_val = y[val_idx]
        
        # Skip if insufficient positives in training or validation
        if y_tr.sum() < 2 or y_val.sum() == 0:
            print(f"  Fold {fold+1}: skipped (insufficient positives)")
            continue
        
        # Train model
        if model_type == 'random_forest':
            model = RandomForestClassifier(
                n_estimators=100,
                max_depth=5,
                class_weight={False: 1, True: class_weight_positive},
                random_state=42
            )
        else:
            model = GradientBoostingClassifier(
                n_estimators=100,
                max_depth=3,
                random_state=42
            )
        
        if model_type == 'gradient_boosting':
            sample_weight = np.where(y_tr, class_weight_positive, 1)
            model.fit(X_tr, y_tr, sample_weight=sample_weight)
        else:
            model.fit(X_tr, y_tr)
        
        # Check model has both classes
        if len(model.classes_) < 2:
            print(f"  Fold {fold+1}: skipped (single class in model)")
            continue
        
        # Get probabilities for positive class
        val_probs = model.predict_proba(X_val)[:, 1]
        
        # Use 5th percentile of positive class probs (more robust than min)
        positive_probs = val_probs[y_val]
        thresh = max(0.05, np.percentile(positive_probs, 5) - 0.01)
        
        # Compute FP rate and FN count at this threshold
        val_pred = val_probs >= thresh
        fn = (y_val.astype(bool) & ~val_pred).sum()
        fp_rate = (~y_val.astype(bool) & val_pred).sum() / max(1, (~y_val.astype(bool)).sum())
        
        cv_thresholds.append(thresh)
        cv_fp_rates.append(fp_rate)
        
        print(f"  Fold {fold+1}: threshold={thresh:.4f}, FN={fn}, FP rate={fp_rate:.1%}")
    
    # Use MEDIAN threshold (more stable than min)
    if cv_thresholds:
        robust_threshold = np.median(cv_thresholds)
    else:
        robust_threshold = 0.10  # Default fallback
    
    print(f"\nRobust threshold (median across folds): {robust_threshold:.4f}")
    
    # Train final model on ALL training data
    print("\nFINAL MODEL TRAINING")
    print("="*60)
    
    if model_type == 'random_forest':
        final_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=5,
            class_weight={False: 1, True: class_weight_positive},
            random_state=42
        )
        final_model.fit(X_scaled, y)
    else:
        final_model = GradientBoostingClassifier(
            n_estimators=100,
            max_depth=3,
            random_state=42
        )
        sample_weight = np.where(y, class_weight_positive, 1)
        final_model.fit(X_scaled, y, sample_weight=sample_weight)
    
    print(f"Model trained on {len(y)} events")
    print(f"Threshold from CV: {robust_threshold:.4f}")
    
    return {
        'model': final_model,
        'scaler': scaler,
        'medians': medians,
        'threshold': robust_threshold,
        'cv_thresholds': cv_thresholds,
        'cv_fp_rates': cv_fp_rates,
        'feature_cols': feature_cols,
    }


In [ ]:
# Define features for CPI → HY model
CPI_HY_FEATURES = [
    'cpi_shock_mom',      # CPI surprise
    'cpi_shock_abs',      # |CPI surprise|
    'yield_volatility',   # Recent yield volatility
    'slope_10y_2y',       # Yield curve slope
    'fed_funds',          # Fed funds rate
    'unemployment',       # Unemployment rate
    'hy_oas_before',      # HY OAS level before
]

# Train model
cpi_hy_result = train_cost_sensitive_model(
    train_df=train_df,
    feature_cols=CPI_HY_FEATURES,
    target_col='is_large_hy_move',
    class_weight_positive=50,
    n_cv_splits=5,
    model_type='random_forest'
)


## Test Evaluation (Ex-Post Diagnostics)

> See [methodology doc](../docs/cpi_hy_unemployment_methodology.md) sections 8-9.

**Note**: The FN=0/FN≤1/FN<1% scenarios below are **ex-post diagnostics** computed on the test set. They show "what threshold *would have been* needed" for each FN constraint on this specific test period. These are NOT deployment thresholds—they help us understand the tradeoff frontier.


In [ ]:
def evaluate_model_triple_thresholds(
    test_df: pd.DataFrame,
    model_result: dict,
    target_col: str,
    target_name: str = "target"
) -> dict:
    """
    Evaluate model on test set with THREE threshold scenarios:
    - FN=0: Catch ALL large moves
    - FN≤1: Allow at most 1 missed
    - FN<1%: Miss less than 1% of positives
    
    Args:
        test_df: Test DataFrame
        model_result: Dictionary from train_cost_sensitive_model
        target_col: Binary target column name
        target_name: Human-readable name for the target
    
    Returns:
        Dictionary with evaluation metrics for all scenarios
    """
    model = model_result['model']
    scaler = model_result['scaler']
    medians = model_result['medians']
    feature_cols = model_result['feature_cols']
    
    # Prepare test features (using TRAINING medians and scaler)
    X_test = test_df[feature_cols].copy()
    X_test = X_test.fillna(medians)
    X_test_scaled = scaler.transform(X_test)
    y_test = test_df[target_col].values
    
    # Get predictions
    test_probs = model.predict_proba(X_test_scaled)[:, 1]
    
    n_positive = y_test.sum()
    n_negative = (~y_test).sum()
    
    # Get sorted positive probabilities
    if n_positive > 0:
        sorted_pos_probs = np.sort(test_probs[y_test])
    else:
        sorted_pos_probs = np.array([0.5])
    
    results = {}
    
    # Scenario 1: FN=0 (catch ALL large moves)
    thresh_fn0 = sorted_pos_probs[0] - 0.001 if n_positive > 0 else 0.5
    pred = test_probs >= thresh_fn0
    results['fn0'] = {
        'threshold': thresh_fn0,
        'TP': (y_test & pred).sum(),
        'FP': (~y_test & pred).sum(),
        'FN': (y_test & ~pred).sum(),
        'TN': (~y_test & ~pred).sum(),
    }
    
    # Scenario 2: FN≤1 (allow at most 1 missed)
    if n_positive > 1:
        thresh_fn1 = sorted_pos_probs[1] - 0.001  # 2nd lowest positive prob
    else:
        thresh_fn1 = thresh_fn0
    thresh_fn1 = max(thresh_fn1, 0.05)
    pred = test_probs >= thresh_fn1
    results['fn_le_1'] = {
        'threshold': thresh_fn1,
        'TP': (y_test & pred).sum(),
        'FP': (~y_test & pred).sum(),
        'FN': (y_test & ~pred).sum(),
        'TN': (~y_test & ~pred).sum(),
    }
    
    # Scenario 3: FN<1% (miss less than 1% of positives)
    max_fn = max(1, int(np.ceil(0.01 * n_positive)))
    if n_positive > max_fn:
        thresh_fn1pct = sorted_pos_probs[max_fn] - 0.001
    else:
        thresh_fn1pct = thresh_fn0
    thresh_fn1pct = max(thresh_fn1pct, 0.05)
    pred = test_probs >= thresh_fn1pct
    results['fn_lt_1pct'] = {
        'threshold': thresh_fn1pct,
        'TP': (y_test & pred).sum(),
        'FP': (~y_test & pred).sum(),
        'FN': (y_test & ~pred).sum(),
        'TN': (~y_test & ~pred).sum(),
    }
    
    # Calculate rates for each scenario
    for key in ['fn0', 'fn_le_1', 'fn_lt_1pct']:
        r = results[key]
        r['fn_rate'] = r['FN'] / max(1, n_positive)
        r['fp_rate'] = r['FP'] / max(1, n_negative)
        r['tn_rate'] = r['TN'] / max(1, n_negative)  # True Negative Rate (Specificity)
        r['precision'] = r['TP'] / max(1, r['TP'] + r['FP'])
        r['recall'] = r['TP'] / max(1, r['TP'] + r['FN'])
    
    results['n_positive'] = n_positive
    results['n_negative'] = n_negative
    results['test_probs'] = test_probs
    
    # Print detailed results
    print("="*80)
    print(f"TEST SET EVALUATION: {target_name}")
    print("="*80)
    print(f"\nTest pool: {n_positive} large moves (+), {n_negative} normal events (-)")
    
    print(f"\n{'Scenario':<12} │ {'Thresh':>7} │ {'TP':>4} {'FP':>4} {'FN':>4} {'TN':>4} │ {'FN%':>6} {'FP%':>6} {'TN%':>6} │ {'Prec':>6}")
    print("─"*80)
    
    for scenario, label in [('fn0', 'FN=0'), ('fn_le_1', 'FN≤1'), ('fn_lt_1pct', 'FN<1%')]:
        r = results[scenario]
        print(f"{label:<12} │ {r['threshold']:>7.4f} │ {r['TP']:>4} {r['FP']:>4} {r['FN']:>4} {r['TN']:>4} │ "
              f"{r['fn_rate']*100:>5.1f}% {r['fp_rate']*100:>5.1f}% {r['tn_rate']*100:>5.1f}% │ {r['precision']:>5.1%}")
    
    # Show impact of relaxing constraint
    fn0_r = results['fn0']
    fn1_r = results['fn_le_1']
    tn_gain = fn1_r['TN'] - fn0_r['TN']
    fp_reduction = fn0_r['FP'] - fn1_r['FP']
    fn_cost = fn1_r['FN'] - fn0_r['FN']
    
    print(f"\n📊 FN=0 → FN≤1 tradeoff:")
    if fp_reduction > 0:
        print(f"   Saves {fp_reduction} false positives, gains {tn_gain} true negatives")
        print(f"   Cost: {fn_cost} additional missed large move(s)")
    else:
        print(f"   No improvement (same threshold)")
    
    # Find structurally unpredictable events
    unpredictable_mask = (test_probs < 0.05) & y_test
    n_unpredictable = unpredictable_mask.sum()
    results['n_unpredictable'] = n_unpredictable
    
    if n_unpredictable > 0:
        print(f"\n⚠ STRUCTURALLY UNPREDICTABLE (p<5% but large move): {n_unpredictable}")
    
    return results


# Backwards compatibility alias
def evaluate_model_on_test(test_df, model_result, target_col, target_name="target", adjust_for_zero_fn=True):
    """Wrapper for backwards compatibility - uses new triple threshold evaluation."""
    return evaluate_model_triple_thresholds(test_df, model_result, target_col, target_name)


In [ ]:
# Evaluate on test set
cpi_hy_eval = evaluate_model_on_test(
    test_df=test_df,
    model_result=cpi_hy_result,
    target_col='is_large_hy_move',
    target_name='CPI → HY OAS Large Move'
)


In [ ]:
# Visualize results with all three scenarios
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Probability distribution
ax1 = axes[0, 0]
y_test = test_df['is_large_hy_move'].values
test_probs = cpi_hy_eval['test_probs']
ax1.hist(test_probs[~y_test], bins=30, alpha=0.6, label='Normal', color='blue')
ax1.hist(test_probs[y_test], bins=15, alpha=0.6, label='Large Move', color='red')

# Show all three thresholds
ax1.axvline(x=cpi_hy_eval['fn0']['threshold'], color='green', linestyle='--', linewidth=2,
            label=f"FN=0 thresh={cpi_hy_eval['fn0']['threshold']:.3f}")
ax1.axvline(x=cpi_hy_eval['fn_le_1']['threshold'], color='orange', linestyle=':', linewidth=2,
            label=f"FN≤1 thresh={cpi_hy_eval['fn_le_1']['threshold']:.3f}")
ax1.set_xlabel('Predicted Probability')
ax1.set_ylabel('Count')
ax1.set_title('CPI→HY: Test Set Probability Distribution')
ax1.legend(fontsize=8)

# Confusion matrices for each scenario
for idx, (scenario, title) in enumerate([('fn0', 'FN=0'), ('fn_le_1', 'FN≤1'), ('fn_lt_1pct', 'FN<1%')]):
    ax = axes[(idx+1)//2, (idx+1)%2]
    r = cpi_hy_eval[scenario]
    cm = np.array([[r['TN'], r['FP']],
                   [r['FN'], r['TP']]])
    im = ax.imshow(cm, cmap='Blues')
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['Pred Normal', 'Pred Large'])
    ax.set_yticklabels(['Actual Normal', 'Actual Large'])
    for i in range(2):
        for j in range(2):
            ax.text(j, i, cm[i, j], ha='center', va='center', fontsize=14, fontweight='bold')
    ax.set_title(f'CPI→HY: {title} (FP={r["FP"]}, TN={r["TN"]})')

plt.tight_layout()
plt.show()

# Print key insight
print("\n📊 CPI→HY KEY INSIGHT:")
fn0 = cpi_hy_eval['fn0']
fn1 = cpi_hy_eval['fn_le_1']
print(f"   At FN=0: FP={fn0['FP']}, TN={fn0['TN']} (TN%={fn0['tn_rate']*100:.1f}%)")
print(f"   At FN≤1: FP={fn1['FP']}, TN={fn1['TN']} (TN%={fn1['tn_rate']*100:.1f}%)")
print(f"   → Relaxing to FN≤1 saves {fn0['FP']-fn1['FP']} false positives")


---

# Part 2: Unemployment Event Family

> See [methodology doc](../docs/cpi_hy_unemployment_methodology.md) for full details.

Same approach as CPI, applied to unemployment releases → 10Y, HY OAS, VIX targets.

**Note**: Unemployment calendar is approximate (first Friday of month). TODO: Replace with official BLS dates.


In [ ]:
def load_or_create_unemployment_calendar(
    panel: pd.DataFrame,
    calendar_path: str = None
) -> pd.DataFrame:
    """
    Load unemployment release calendar or create an approximation.
    
    The unemployment report (Jobs Report) is released on the first Friday of each month.
    
    Args:
        panel: Daily FRED panel with 'unemployment' column
        calendar_path: Path to unemployment calendar CSV (optional)
    
    Returns:
        DataFrame with release_date, data_period, reported_unemployment, 
        previous_unemployment, unemployment_surprise
    """
    # Try to load from file first
    if calendar_path:
        try:
            cal = pd.read_csv(calendar_path, parse_dates=['release_date'])
            logger.info(f"Loaded unemployment calendar from {calendar_path}")
            return cal
        except Exception as e:
            logger.warning(f"Could not load unemployment calendar: {e}. Creating approximation.")
    
    # Create approximate calendar based on first Friday of each month
    # TODO: Replace with actual BLS release times from official source
    logger.info("Creating approximate unemployment release calendar (first Friday of each month)")
    
    panel = panel.copy()
    panel['date'] = pd.to_datetime(panel['date'])
    panel = panel.sort_values('date')
    
    # Get unique months with unemployment data
    panel_with_unemp = panel.dropna(subset=['unemployment'])
    
    # Find first Friday of each month
    releases = []
    
    for year in panel_with_unemp['date'].dt.year.unique():
        for month in range(1, 13):
            # Find the first Friday of this month
            first_day = pd.Timestamp(year=year, month=month, day=1)
            # Days until Friday (Friday = 4)
            days_until_friday = (4 - first_day.dayofweek) % 7
            first_friday = first_day + pd.Timedelta(days=days_until_friday)
            
            # Get unemployment values
            # The release on first Friday of month M reports data for month M-1
            if month == 1:
                data_period = f"{year-1}-12"
            else:
                data_period = f"{year}-{month-1:02d}"
            
            # Find nearest panel date
            panel_dates = panel_with_unemp[panel_with_unemp['date'] <= first_friday]
            if panel_dates.empty:
                continue
            
            # Get unemployment on release date
            release_row = panel_with_unemp[panel_with_unemp['date'] == first_friday]
            if release_row.empty:
                # Try nearby dates
                for offset in [1, -1, 2, -2, 3]:
                    candidate = first_friday + pd.Timedelta(days=offset)
                    release_row = panel_with_unemp[panel_with_unemp['date'] == candidate]
                    if not release_row.empty:
                        first_friday = candidate
                        break
            
            if release_row.empty:
                continue
            
            reported = release_row.iloc[0]['unemployment']
            
            # Get previous month's unemployment
            prev_rows = panel_with_unemp[panel_with_unemp['date'] < first_friday - pd.Timedelta(days=20)]
            if prev_rows.empty:
                previous = np.nan
            else:
                previous = prev_rows.iloc[-1]['unemployment']
            
            # Compute surprise (simple change)
            surprise = reported - previous if pd.notna(previous) else np.nan
            
            releases.append({
                'release_date': first_friday,
                'data_period': data_period,
                'reported_unemployment': reported,
                'previous_unemployment': previous,
                'unemployment_surprise': surprise,
            })
    
    cal = pd.DataFrame(releases)
    cal = cal.dropna(subset=['reported_unemployment', 'previous_unemployment'])
    cal = cal.sort_values('release_date').reset_index(drop=True)
    
    logger.info(f"Created unemployment calendar: {len(cal)} releases")
    
    return cal


In [ ]:
# Create/load unemployment calendar
unemp_calendar = load_or_create_unemployment_calendar(panel)
print(f"Unemployment releases: {len(unemp_calendar)}")
print(f"Date range: {unemp_calendar['release_date'].min().date()} to {unemp_calendar['release_date'].max().date()}")
unemp_calendar.head(10)


## Dataset Builder
Reusable function for any target (10Y, HY OAS, VIX).


In [ ]:
def build_unemployment_event_dataset(
    panel: pd.DataFrame,
    unemp_calendar: pd.DataFrame,
    target_series: str = 'y_10y',
    window_days: int = 0,
    min_history_days: int = 30,
    yield_vol_window: int = 20
) -> pd.DataFrame:
    """
    Build event-level dataset for unemployment → target large move detection.
    
    Args:
        panel: Daily FRED panel
        unemp_calendar: Unemployment release calendar
        target_series: Target series to detect moves in ('y_10y', 'hy_oas', 'vix')
        window_days: Days after release to measure change (0 = same day)
        min_history_days: Minimum days of history required
        yield_vol_window: Rolling window for volatility calculation
    
    Returns:
        DataFrame with one row per unemployment event
    """
    # Map target_series to panel column
    target_col_map = {
        'y_10y': 'y_10y', 'DGS10': 'y_10y',
        'hy_oas': 'hy_oas', 'BAMLH0A0HYM2': 'hy_oas',
        'vix': 'vix', 'VIXCLS': 'vix',
    }
    target_col = target_col_map.get(target_series, target_series)
    
    if target_col not in panel.columns:
        raise ValueError(f"Need {target_col} in panel for unemployment→{target_series} model.")
    
    panel = panel.copy()
    panel['date'] = pd.to_datetime(panel['date'])
    panel = panel.sort_values('date').reset_index(drop=True)
    
    unemp_calendar = unemp_calendar.copy()
    unemp_calendar['release_date'] = pd.to_datetime(unemp_calendar['release_date'])
    
    events = []
    
    for idx, rel in unemp_calendar.iterrows():
        rel_date = rel['release_date']
        data_period = rel['data_period']
        
        # Find panel row for release date
        rel_day = panel[panel['date'] == rel_date]
        if rel_day.empty:
            for offset in [1, -1, 2, -2, 3, -3]:
                candidate = rel_date + pd.Timedelta(days=offset)
                rel_day = panel[panel['date'] == candidate]
                if not rel_day.empty:
                    rel_date = candidate
                    break
        
        if rel_day.empty:
            continue
        
        rel_row = rel_day.iloc[0]
        
        # Get previous trading day
        prev_days = panel[panel['date'] < rel_date]
        if len(prev_days) < min_history_days:
            continue
        prev_row = prev_days.iloc[-1]
        
        # Get target before and after
        target_before = prev_row.get(target_col)
        
        if window_days == 0:
            target_after = rel_row.get(target_col)
        else:
            future_date = rel_date + pd.Timedelta(days=window_days)
            future_rows = panel[panel['date'] >= future_date]
            if future_rows.empty:
                continue
            target_after = future_rows.iloc[0].get(target_col)
        
        if pd.isna(target_before) or pd.isna(target_after):
            continue
        
        target_change = target_after - target_before
        
        # Calculate volatility (trailing window)
        recent = prev_days.tail(yield_vol_window)
        if len(recent) > 5:
            vol_vals = recent[target_col].dropna().values
            volatility = np.std(np.diff(vol_vals)) if len(vol_vals) > 5 else np.nan
        else:
            volatility = np.nan
        
        event = {
            'date': rel_date,
            'data_period': data_period,
            'unemployment_surprise': rel['unemployment_surprise'],
            'unemployment_surprise_abs': abs(rel['unemployment_surprise']) if pd.notna(rel['unemployment_surprise']) else np.nan,
            'reported_unemployment': rel['reported_unemployment'],
            'previous_unemployment': rel['previous_unemployment'],
            f'{target_col}_volatility': volatility,
            'slope_10y_2y': prev_row.get('slope_10y_2y'),
            'fed_funds': prev_row.get('fed_funds'),
            f'{target_col}_before': target_before,
            'hy_oas_before': prev_row.get('hy_oas'),
            'stlfsi': prev_row.get('stlfsi', np.nan),
            'vix_before': prev_row.get('vix', np.nan),
            f'{target_col}_change': target_change,
            f'{target_col}_change_abs': abs(target_change),
        }
        events.append(event)
    
    df = pd.DataFrame(events)
    if df.empty:
        logger.warning(f"No unemployment→{target_col} events created.")
        return df
    
    df = df.dropna(subset=[f'{target_col}_change'])
    df = df.sort_values('date').reset_index(drop=True)
    
    logger.info(f"Built unemployment→{target_col} dataset: {len(df)} events")
    return df


In [ ]:
# Build datasets for each target
unemp_10y_df = build_unemployment_event_dataset(panel, unemp_calendar, target_series='y_10y')
unemp_hy_df = build_unemployment_event_dataset(panel, unemp_calendar, target_series='hy_oas')
unemp_vix_df = build_unemployment_event_dataset(panel, unemp_calendar, target_series='vix')

print(f"\nUnemployment → 10Y Events: {len(unemp_10y_df)}")
print(f"Unemployment → HY OAS Events: {len(unemp_hy_df)}")
print(f"Unemployment → VIX Events: {len(unemp_vix_df)}")


## Train/Evaluate Models
Same methodology as CPI → HY, applied to each target.


In [ ]:
def train_unemployment_model_for_target(
    df: pd.DataFrame,
    target_col: str,
    test_size: float = 0.30,
    threshold_percentile: float = 90.0,
    class_weight_positive: int = 50,
    model_type: str = 'random_forest'
) -> dict:
    """
    Train and evaluate a cost-sensitive model for unemployment → target large move.
    """
    target_name = target_col.replace('_change', '')
    
    print("\n" + "="*70)
    print(f"UNEMPLOYMENT → {target_name.upper()} MODEL")
    print("="*70)
    
    # Chronological split
    n_total = len(df)
    n_test = int(n_total * test_size)
    n_train = n_total - n_test
    
    unemp_train_df = df.iloc[:n_train].copy()
    unemp_test_df = df.iloc[n_train:].copy()
    
    print(f"\nTrain: {len(unemp_train_df)} events ({unemp_train_df['date'].min().date()} to {unemp_train_df['date'].max().date()})")
    print(f"Test:  {len(unemp_test_df)} events ({unemp_test_df['date'].min().date()} to {unemp_test_df['date'].max().date()})")
    
    # Compute threshold from training data
    threshold = compute_large_move_threshold_from_train(unemp_train_df, target_col, threshold_percentile)
    
    # Create binary labels
    is_large_col = f'is_large_{target_name}_move'
    unemp_train_df[is_large_col] = unemp_train_df[target_col].abs() >= threshold
    unemp_test_df[is_large_col] = unemp_test_df[target_col].abs() >= threshold
    
    print(f"\nTrain large moves: {unemp_train_df[is_large_col].sum()} ({unemp_train_df[is_large_col].mean()*100:.1f}%)")
    print(f"Test large moves:  {unemp_test_df[is_large_col].sum()} ({unemp_test_df[is_large_col].mean()*100:.1f}%)")
    
    # Define features
    feature_cols = [
        'unemployment_surprise',
        'unemployment_surprise_abs',
        f'{target_name}_volatility',
        'slope_10y_2y',
        'fed_funds',
        f'{target_name}_before',
    ]
    
    # Only include columns that exist
    feature_cols = [c for c in feature_cols if c in unemp_train_df.columns]
    
    # Add HY OAS before if available and not the target
    if 'hy_oas_before' in unemp_train_df.columns and target_name != 'hy_oas':
        feature_cols.append('hy_oas_before')
    
    print(f"\nFeatures: {feature_cols}")
    
    # Train model
    model_result = train_cost_sensitive_model(
        train_df=unemp_train_df,
        feature_cols=feature_cols,
        target_col=is_large_col,
        class_weight_positive=class_weight_positive,
        n_cv_splits=5,
        model_type=model_type
    )
    
    # Evaluate on test set
    eval_result = evaluate_model_on_test(
        test_df=unemp_test_df,
        model_result=model_result,
        target_col=is_large_col,
        target_name=f"Unemployment → {target_name.upper()} Large Move"
    )
    
    return {
        'model_result': model_result,
        'eval_result': eval_result,
        'train_df': unemp_train_df,
        'test_df': unemp_test_df,
        'large_move_threshold': threshold,
        'target_col': target_col,
        'is_large_col': is_large_col,
    }


In [ ]:
# Train model for Unemployment → 10Y
unemp_10y_result = train_unemployment_model_for_target(
    df=unemp_10y_df,
    target_col='y_10y_change',
    test_size=0.30,
    threshold_percentile=85.0  # Using 85th pctl for more events
)


In [ ]:
# Train model for Unemployment → HY OAS
unemp_hy_result = train_unemployment_model_for_target(
    df=unemp_hy_df,
    target_col='hy_oas_change',
    test_size=0.30,
    threshold_percentile=85.0  # Using 85th pctl for more events
)


In [ ]:
# Train model for Unemployment → VIX
unemp_vix_result = train_unemployment_model_for_target(
    df=unemp_vix_df,
    target_col='vix_change',
    test_size=0.30,
    threshold_percentile=85.0  # Using 85th pctl for more events
)


---
# Results Summary (Ex-Post Diagnostics)

> See [methodology doc](../docs/cpi_hy_unemployment_methodology.md) section 11.

**Note**: Tables below show ex-post thresholds that *would have achieved* FN=0/1/etc on this test period. The CV-selected deployment threshold (Section 7) is computed separately from training data only.


In [ ]:
# Comprehensive Summary Table with All Three Scenarios
print("\n" + "="*100)
print("COMPREHENSIVE SUMMARY: ALL MODELS WITH THREE THRESHOLD SCENARIOS")
print("="*100)

# Collect all results
all_results = {
    'CPI→HY': {
        'results': cpi_hy_eval,
        'train_n': len(train_df),
        'test_n': len(test_df),
        'thresh': LARGE_HY_THRESHOLD
    },
    'Unemp→10Y': {
        'results': unemp_10y_result['eval_result'],
        'train_n': len(unemp_10y_result['train_df']),
        'test_n': len(unemp_10y_result['test_df']),
        'thresh': unemp_10y_result['large_move_threshold']
    },
    'Unemp→HY': {
        'results': unemp_hy_result['eval_result'],
        'train_n': len(unemp_hy_result['train_df']),
        'test_n': len(unemp_hy_result['test_df']),
        'thresh': unemp_hy_result['large_move_threshold']
    },
    'Unemp→VIX': {
        'results': unemp_vix_result['eval_result'],
        'train_n': len(unemp_vix_result['train_df']),
        'test_n': len(unemp_vix_result['test_df']),
        'thresh': unemp_vix_result['large_move_threshold']
    },
}

# Print summary for each scenario
for scenario, scenario_label in [('fn0', 'FN=0 (Never miss a large move)'), 
                                  ('fn_le_1', 'FN≤1 (Allow at most 1 missed)'),
                                  ('fn_lt_1pct', 'FN<1% (Miss less than 1%)')]:
    print(f"\n┌{'─'*96}┐")
    print(f"│{scenario_label:^96}│")
    print(f"├{'─'*96}┤")
    print(f"│ {'Model':<12} │ {'Pool':<12} │ {'TP':>4} {'FP':>4} {'FN':>4} {'TN':>4} │ {'FN%':>6} {'FP%':>6} {'TN%':>6} │ {'Prec':>6} │")
    print(f"├{'─'*96}┤")
    
    for name, data in all_results.items():
        res = data['results']
        r = res[scenario]
        n_pos = res['n_positive']
        n_neg = res['n_negative']
        pool_str = f"{n_pos}+ / {n_neg}-"
        
        print(f"│ {name:<12} │ {pool_str:<12} │ {r['TP']:>4} {r['FP']:>4} {r['FN']:>4} {r['TN']:>4} │ "
              f"{r['fn_rate']*100:>5.1f}% {r['fp_rate']*100:>5.1f}% {r['tn_rate']*100:>5.1f}% │ {r['precision']:>5.1%} │")
    
    print(f"└{'─'*96}┘")

# Print tradeoff analysis
print("\n" + "="*80)
print("TRADEOFF ANALYSIS: FN=0 → FN≤1")
print("="*80)
print(f"{'Model':<12} │ {'FP Saved':>10} │ {'TN Gained':>10} │ {'FN Cost':>10} │ {'Recommendation':<20}")
print("─"*70)

for name, data in all_results.items():
    res = data['results']
    fn0 = res['fn0']
    fn1 = res['fn_le_1']
    
    fp_saved = fn0['FP'] - fn1['FP']
    tn_gained = fn1['TN'] - fn0['TN']
    fn_cost = fn1['FN'] - fn0['FN']
    
    if fp_saved > 5:
        rec = "Use FN≤1 (big gain)"
    elif fp_saved > 0:
        rec = "Consider FN≤1"
    else:
        rec = "Use FN=0 (same)"
    
    print(f"{name:<12} │ {fp_saved:>10} │ {tn_gained:>10} │ {fn_cost:>10} │ {rec:<20}")

print("\n" + "="*80)
print("LEGEND")
print("="*80)
print("""
Pool   : Number of actual Large (+) vs Not-Large (-) moves in test set
TP     : True Positives (correctly flagged large moves)
FP     : False Positives (incorrectly flagged as large - cost: unnecessary hedging)
FN     : False Negatives (missed large moves - cost: unhedged exposure!)
TN     : True Negatives (correctly rejected non-large events - benefit: no false alarm)
FN%    : False Negative Rate = FN / (TP + FN) - want this LOW
FP%    : False Positive Rate = FP / (FP + TN) - tradeoff with FN
TN%    : True Negative Rate = TN / (FP + TN) = 100% - FP% - shows filtering power
Prec   : Precision = TP / (TP + FP) - what fraction of alerts are real
""")


---

# Conclusions

> 📚 **Full methodology**: [`docs/cpi_hy_unemployment_methodology.md`](../docs/cpi_hy_unemployment_methodology.md)  
> 🔧 **Grid search module**: `src/models/event_grid_search.py`

## Exhaustive Grid Search (19,602 configs tested)

**Primary constraint**: FN ≤ 1% or ≤ 5% (catch almost all large moves)  
**Secondary**: Maximize TN/FP ratio

### Best Configs with FN ≤ 1% (100% Recall)

| Model | Config | TP | FP | FN | TN | TN/FP |
|-------|--------|-----|-----|-----|-----|-------|
| **CPI→HY** | LogReg, w=50, 15bp, @0.80 | 8 | 19 | 0 | 77 | **4.05x** |
| **Unemp→VIX** | RF_shallow, w=50, 2.5pt, @0.20 | 15 | 73 | 0 | 42 | 0.58x |

### Best Configs with FN ≤ 5% (95%+ Recall)

| Model | Config | TP | FP | FN | TN | TN/FP |
|-------|--------|-----|-----|-----|-----|-------|
| **CPI→HY** | LogReg, w=50, 15bp, @0.80 | 8 | 19 | 0 | 77 | **4.05x** |
| **Unemp→VIX** | GB, w=20, 2.0pt, @0.05 | 20 | 48 | 1 | 61 | **1.27x** |

## Key Findings

1. **CPI→HY**: LogReg with high weight (50) is best - AUC=0.911, TN/FP=4.05x
2. **Unemp→VIX**: Trade-off between FN≤1% (100% recall, TN/FP=0.58x) vs FN≤5% (95% recall, TN/FP=1.27x)
3. **Higher thresholds** (15bp, 2-2.5pt) enable better TN/FP ratios

## TODOs

- [ ] Consensus-based surprise (actual vs. analyst)
- [ ] Official BLS calendar
- [ ] More features (VIX term structure, FedWatch)
